<hr></hr>
Change this parameter to use it on your own file

In [6]:
ifile = 'test_annovar.hg19.tsv'

<hr></hr>

##Parsing ANNOVAR
A new approach @CINECA


* Using [mETL](https://github.com/ceumicrodata/mETL) package
* Configuration file is in [YAML format](http://pyyaml.org/wiki/PyYAMLDocumentation) through python library

### Define fields to be parsed

In [7]:
fields = [
# Chromosome
{'map': 'Chr', 'name': 'chromosome', 'type': 'String'},
# Start
{'map': 'Start', 'name': 'position', 'type': 'Float'},
]

### YAML configuration with python code

Configuration for source and target after decisions on fields

In [8]:
# Create YAML configuration file automatically
import yaml
yfile = 'test.yml'
sqlfile = 'test.db'
table = 'test'
# Init python dict
data = {}
# Input
data['source'] = { 'resource': ifile, 'fields': fields, 'source': 'TSV', 'headerRow': 0, 'skipRows': 1}
# Output
data['target'] = {
    'type': 'Database', 'createTable': 'true', 'replaceTable': 'true', 'truncateTable': 'true',
    'table': table, 'url': 'sqlite:///' + sqlfile,
}

Write the files and check content

In [9]:
# Write the file
with open(yfile, 'w') as outfile:
    outfile.write( yaml.dump(data, default_flow_style=False) )

#!head {yfile} -n 3
out = !! wc -l {yfile}
print "Your configuration inside file '" + yfile + "' reaches " + out.s.split(" ")[0] + " lines"

Your configuration inside file 'test.yml' reaches 19 lines


### Execute the parser

The following use the *metl* package to read the variants annotated and save it into a file database

In [10]:
# Execute via ipython
! metl {yfile} 

### Use the database

Read data in python and manipulate

In [14]:
# init
import sqlite3 as lite
conn = None
# connect
try:
    conn = lite.connect(sqlfile)    
    cur = conn.cursor()     
except lite.Error, e:
    raise BaseException("No sql lite connection", e)
finally:
    print "Connected"

Connected


In [17]:
# Query
query = "SELECT * FROM " + table
cur.execute(query)
for i in cur.fetchall():
    print i


(1, u'chr1', 10385471)
(2, u'chr1', 17722363)
(3, u'chr1', 45796269)
(4, u'chr1', 45798555)
(5, u'chr1', 121280613)
(6, u'chr1', 150860471)
(7, u'chr1', 161332092)
(8, u'chr1', 193094375)
(9, u'chr1', 193107187)
(10, u'chr1', 222045446)
(11, u'chr1', 228997835)
(12, u'chr1', 241663902)
(13, u'chr1', 241663902)
(14, u'chr1', 241663941)
(15, u'chr2', 8178735)
(16, u'chr2', 15782471)
(17, u'chr2', 29416572)
(18, u'chr2', 29419591)
(19, u'chr2', 29449819)
(20, u'chr2', 29455267)
(21, u'chr2', 29543663)
(22, u'chr2', 29940529)
(23, u'chr2', 30143499)
(24, u'chr2', 43553949)
(25, u'chr2', 46537604)
(26, u'chr2', 47635523)
(27, u'chr2', 47702452)
(28, u'chr2', 48010558)
(29, u'chr2', 48010654)
(30, u'chr2', 48018081)
(31, u'chr2', 48023115)
(32, u'chr2', 48027755)
(33, u'chr2', 48030838)
(34, u'chr2', 48032740)
(35, u'chr2', 48032875)
(36, u'chr2', 48033551)
(37, u'chr2', 48033891)
(38, u'chr2', 58388696)
(39, u'chr2', 58453843)
(40, u'chr2', 61066666)
(41, u'chr2', 63131731)
(42, u'chr2', 85

<hr></hr>


* finale notes*

- transformation with python function
- manipulation (filters)
- convert from db to excel for download ;)

In [13]:
# The end, release resources
if conn:
    conn.close()